Te ayudo a instalar n8n y crear un workflow completo con Gemini como chatbot de programación. Aquí está el paso a paso detallado:

## 1. Instalación de n8n

```bash
# Actualizar el sistema
sudo apt update && sudo apt upgrade -y

# Instalar Node.js (n8n requiere Node.js 18+)
curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
sudo apt-get install -y nodejs

# Verificar instalación
node --version
npm --version

# Instalar n8n globalmente
sudo npm install -g n8n

# Crear directorio para n8n
mkdir ~/n8n-projects
cd ~/n8n-projects
```

## 2. Configuración inicial de n8n

```bash
# Crear archivo de configuración
cat > ~/n8n-projects/ecosystem.config.js << 'EOF'
module.exports = {
  apps: [{
    name: 'n8n',
    script: 'n8n',
    instances: 1,
    autorestart: true,
    watch: false,
    max_memory_restart: '1G',
    env: {
      N8N_BASIC_AUTH_ACTIVE: true,
      N8N_BASIC_AUTH_USER: 'admin',
      N8N_BASIC_AUTH_PASSWORD: 'password123',
      N8N_ENCRYPTION_KEY: 'gemini-chatbot-key-2024',
      NODE_ENV: 'production'
    }
  }]
};
EOF

# Instalar PM2 para gestionar el proceso
sudo npm install -g pm2

# Iniciar n8n con PM2
pm2 start ecosystem.config.js
pm2 save
pm2 startup
```

## 3. Configurar Gemini API

Primero, obtén tu API key gratuita de Gemini:
1. Ve a [Google AI Studio](https://aistudio.google.com/)
2. Inicia sesión con tu cuenta Google
3. Crea una nueva API key
4. Cópiala para usarla después

## 4. Workflow de n8n para Chatbot Gemini

### Paso 4.1: Acceder a n8n
```bash
# Verificar que n8n esté corriendo
pm2 status

# Abre tu navegador y ve a:
# http://localhost:5678
# Usuario: admin
# Contraseña: password123
```

### Paso 4.2: Crear el workflow

**Nodos necesarios:**
1. **Webhook** (para recibir mensajes)
2. **Code** (para gestión de memoria conversacional)
3. **HTTP Request** (para llamar a Gemini API)
4. **Set** (para preparar respuestas)
5. **SQLite** (para guardar conversaciones)

### Configuración paso a paso:

#### **Nodo 1: Webhook (Trigger)**
- **Webhook Method**: POST
- **Path**: `/gemini-chat`
- **Response Mode**: Last Node

#### **Nodo 2: Code - Procesar Entrada y Memoria**
```javascript
// Configurar memoria conversacional
const userId = $input.body.userId || 'default-user';
const userMessage = $input.body.message;
const sessionId = $input.body.sessionId || 'default-session';

// Recuperar historial de conversación
const existingHistory = await $workflow.getStaticData('user', userId);
let conversationHistory = existingHistory.history || [];

// Mantener solo las últimas 10 interacciones para contexto
if (conversationHistory.length > 10) {
  conversationHistory = conversationHistory.slice(-10);
}

// Preparar contexto para Gemini
const context = conversationHistory.map(entry => 
  `${entry.role}: ${entry.content}`
).join('\n');

const newUserEntry = {
  role: 'user',
  content: userMessage,
  timestamp: new Date().toISOString()
};

return {
  userId,
  sessionId,
  userMessage,
  conversationHistory,
  context,
  newUserEntry
};
```

#### **Nodo 3: HTTP Request - Llamar a Gemini API**
- **Method**: POST
- **URL**: `https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=TU_API_KEY_AQUI`
- **Headers**:
  - `Content-Type: application/json`
- **Body** (JSON):
```json
{
  "contents": [
    {
      "parts": [
        {
          "text": "Eres un asistente experto en programación. Responde preguntas sobre código, algoritmos, lenguajes de programación y mejores prácticas. Mantén respuestas claras y didácticas.\n\nHistorial de conversación:\n{{ $json.context }}\n\nUsuario: {{ $json.userMessage }}\nAsistente:"
        }
      ]
    }
  ],
  "generationConfig": {
    "temperature": 0.7,
    "maxOutputTokens": 1000
  }
}
```

#### **Nodo 4: Code - Procesar Respuesta y Actualizar Memoria**
```javascript
// Extraer respuesta de Gemini
const geminiResponse = $input.json();
const assistantReply = geminiResponse.candidates[0].content.parts[0].text;

// Actualizar historial de conversación
const userId = $json.userId;
const newUserEntry = $json.newUserEntry;
const existingHistory = await $workflow.getStaticData('user', userId);
let conversationHistory = existingHistory.history || [];

// Agregar nueva interacción
conversationHistory.push(newUserEntry);
conversationHistory.push({
  role: 'assistant',
  content: assistantReply,
  timestamp: new Date().toISOString()
});

// Guardar historial actualizado
await $workflow.setStaticData('user', userId, {
  history: conversationHistory,
  lastActivity: new Date().toISOString()
});

return {
  userId,
  originalMessage: $json.userMessage,
  assistantReply,
  timestamp: new Date().toISOString(),
  historyLength: conversationHistory.length
};
```

#### **Nodo 5: SQLite - Guardar Conversación en Base de Datos**

Primero instala SQLite en n8n:
```bash
cd ~/n8n-projects
npm install n8n-nodes-sqlite
```

**Configuración del nodo SQLite:**
- **Operation**: Insert
- **Table**: conversations
- **Columns**:
  - userId: `{{ $json.userId }}`
  - sessionId: `{{ $json.sessionId }}`
  - user_message: `{{ $json.originalMessage }}`
  - assistant_reply: `{{ $json.assistantReply }}`
  - timestamp: `{{ $json.timestamp }}`

#### **Nodo 6: Set - Preparar Respuesta Final**
- **Add Value**:
  - **Name**: response
  - **Value**: `{{ $json.assistantReply }}`

## 5. Configuración de la Base de Datos SQLite

Crea la base de datos y tabla:
```bash
# Instalar SQLite3
sudo apt install sqlite3

# Crear base de datos
sqlite3 ~/n8n-projects/chatbot.db

# En la consola SQLite:
CREATE TABLE conversations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    userId TEXT NOT NULL,
    sessionId TEXT NOT NULL,
    user_message TEXT NOT NULL,
    assistant_reply TEXT NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_userId ON conversations(userId);
CREATE INDEX idx_timestamp ON conversations(timestamp);

.quit
```

## 6. Probar el Workflow

Usa curl para probar el chatbot:

```bash
curl -X POST http://localhost:5678/webhook/gemini-chat \
  -H "Content-Type: application/json" \
  -d '{
    "userId": "alumno1",
    "sessionId": "sesion-programacion",
    "message": "¿Qué es Python y para qué se usa?"
  }'
```

## 7. Configuración para WhatsApp (Opcional)

Para integrar WhatsApp necesitarías:
1. **Twilio API** (tiene capa gratuita)
2. **Nodo Twilio** en n8n

**Instalar nodo Twilio:**
```bash
cd ~/n8n-projects
npm install n8n-nodes-twilio
```

## 8. Script de Gestión Completo

Crea un script de gestión:

```bash
cat > ~/n8n-projects/manage-chatbot.sh << 'EOF'
#!/bin/bash

case $1 in
    start)
        pm2 start ecosystem.config.js
        echo "n8n iniciado"
        ;;
    stop)
        pm2 stop n8n
        echo "n8n detenido"
        ;;
    status)
        pm2 status
        ;;
    logs)
        pm2 logs n8n
        ;;
    backup-db)
        cp ~/n8n-projects/chatbot.db ~/n8n-projects/backup/chatbot_$(date +%Y%m%d_%H%M%S).db
        echo "Base de datos respaldada"
        ;;
    *)
        echo "Uso: $0 {start|stop|status|logs|backup-db}"
        ;;
esac
EOF

chmod +x ~/n8n-projects/manage-chatbot.sh
```

## 9. Verificar Funcionamiento

```bash
# Verificar servicios
./manage-chatbot.sh status

# Probar con preguntas de programación
curl -X POST http://localhost:5678/webhook/gemini-chat \
  -H "Content-Type: application/json" \
  -d '{
    "userId": "test-user",
    "sessionId": "test-session", 
    "message": "Explícame qué son las funciones en JavaScript"
  }'
```

## Características del Chatbot:

✅ **Memoria conversacional** - Recuerda el contexto de la conversación  
✅ **Base de datos** - Guarda todas las interacciones  
✅ **Gratuito** - Usa APIs gratuitas  
✅ **Especializado en programación** - Optimizado para preguntas técnicas  
✅ **Fácil de explicar** - Workflow visual para enseñanza  

¿Te gustaría que profundice en alguna parte específica o necesitas ayuda con la configuración de algún nodo?